# Lesson 6: Essay Writer

In [29]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
from tavily import TavilyClient


load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

model_id = "llama-3.1-8b-instant" # 	qwen/qwen3-32b  openai/gpt-oss-20b llama-3.1-8b-instant
    
llm = ChatGroq(model=model_id,
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    verbose=1)



In [30]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ChatMessage

# from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.checkpoint.memory import InMemorySaver

# memory = SqliteSaver.from_conn_string(":memory:")
memory = InMemorySaver()

In [31]:
class AgentState(TypedDict):
    task: str
    plan: str
    draft: str
    critique: str
    content: List[str]
    revision_number: int
    max_revisions: int

In [32]:
PLAN_PROMPT = """You are an expert writer tasked with writing a high level outline of an essay. \
Write such an outline for the user provided topic. Give an outline of the essay along with any relevant notes \
or instructions for the sections."""

In [33]:
WRITER_PROMPT = """You are an essay assistant tasked with writing excellent 5-paragraph essays.\
Generate the best essay possible for the user's request and the initial outline. \
If the user provides critique, respond with a revised version of your previous attempts. \
Utilize all the information below as needed: 

------

{content}"""

In [34]:
REFLECTION_PROMPT = """You are a teacher grading an essay submission. \
Generate critique and recommendations for the user's submission. \
Provide detailed recommendations, including requests for length, depth, style, etc."""

In [35]:
RESEARCH_PLAN_PROMPT = """You are a researcher charged with providing information that can \
be used when writing the following essay. Generate a list of search queries that will gather \
any relevant information. Only generate 3 queries max."""


In [36]:
RESEARCH_CRITIQUE_PROMPT = """You are a researcher charged with providing information that can \
be used when making any requested revisions (as outlined below). \
Generate a list of search queries that will gather any relevant information. Only generate 3 queries max."""


In [37]:
from langchain_core.pydantic_v1 import BaseModel

class Queries(BaseModel):
    queries: List[str]

In [38]:
from tavily import TavilyClient
import os
tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

In [39]:
def plan_node(state: AgentState):
    messages = [
        SystemMessage(content=PLAN_PROMPT), 
        HumanMessage(content=state['task'])
    ]
    response = llm.invoke(messages)
    return {"plan": response.content}

In [40]:
def research_plan_node(state: AgentState):
    queries = llm.with_structured_output(Queries).invoke([
        SystemMessage(content=RESEARCH_PLAN_PROMPT),
        HumanMessage(content=state['task'])
    ])
    content = state['content'] or []
    for q in queries.queries:
        response = tavily.search(query=q, max_results=2)
        for r in response['results']:
            content.append(r['content'])
    return {"content": content}

In [41]:
def generation_node(state: AgentState):
    content = "\n\n".join(state['content'] or [])
    user_message = HumanMessage(
        content=f"{state['task']}\n\nHere is my plan:\n\n{state['plan']}")
    messages = [
        SystemMessage(
            content=WRITER_PROMPT.format(content=content)
        ),
        user_message
        ]
    response = llm.invoke(messages)
    return {
        "draft": response.content, 
        "revision_number": state.get("revision_number", 1) + 1
    }


In [42]:
def reflection_node(state: AgentState):
    messages = [
        SystemMessage(content=REFLECTION_PROMPT), 
        HumanMessage(content=state['draft'])
    ]
    response = llm.invoke(messages)
    return {"critique": response.content}

In [43]:
def research_critique_node(state: AgentState):
    queries = llm.with_structured_output(Queries).invoke([
        SystemMessage(content=RESEARCH_CRITIQUE_PROMPT),
        HumanMessage(content=state['critique'])
    ])
    content = state['content'] or []
    for q in queries.queries:
        response = tavily.search(query=q, max_results=2)
        for r in response['results']:
            content.append(r['content'])
    return {"content": content}

In [44]:
def should_continue(state):
    if state["revision_number"] > state["max_revisions"]:
        return END
    return "reflect"

In [45]:
builder = StateGraph(AgentState)

In [46]:
builder.add_node("planner", plan_node)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_node("research_plan", research_plan_node)
builder.add_node("research_critique", research_critique_node)

In [47]:
builder.set_entry_point("planner")

In [48]:
builder.add_conditional_edges(
    "generate", 
    should_continue, 
    {END: END, "reflect": "reflect"}
)


In [49]:
builder.add_edge("planner", "research_plan")
builder.add_edge("research_plan", "generate")

builder.add_edge("reflect", "research_critique")
builder.add_edge("research_critique", "generate")

In [50]:
graph = builder.compile(checkpointer=memory)

In [ ]:
class AgentState(TypedDict):
    task: str
    plan: str
    draft: str
    critique: str
    content: List[str]
    revision_number: int
    max_revisions: int

In [53]:
thread = {"configurable": {"thread_id": "2"}}
for s in graph.stream({
    'task': "what is the difference between langchain and langsmith",
    'plan': '',
    'draft': '',
    'critique': '',
    'content': '',
    "max_revisions": 2,
    "revision_number": 1,
}, thread):
    print(s)

{'planner': {'plan': "Here's a high-level outline for an essay on the differences between LangChain and LangSmith:\n\n**I. Introduction**\n- Briefly introduce the topic of AI-powered language models and their applications\n- Mention LangChain and LangSmith as two prominent tools in this space\n- Thesis statement: While both LangChain and LangSmith are AI-powered language tools, they differ in their approach, functionality, and use cases.\n\n**II. Overview of LangChain**\n- Define LangChain and its core features\n- Explain how LangChain enables users to build custom AI applications using a modular architecture\n- Highlight LangChain's strengths, such as its flexibility and scalability\n- Relevant notes: LangChain is an open-source framework that allows users to create custom AI models and integrate them with various data sources.\n\n**III. Overview of LangSmith**\n- Define LangSmith and its core features\n- Explain how LangSmith uses a combination of natural language processing (NLP) an

In [56]:
print(s['generate']['draft'])

Here's a draft essay based on your outline:

**The Differences between LangChain and LangSmith: A Comparative Analysis**

The rise of artificial intelligence (AI) has led to the development of various tools and frameworks that enable users to build custom AI applications. Two prominent tools in this space are LangChain and LangSmith, both of which have gained significant attention in recent years. While both tools are AI-powered language tools, they differ in their approach, functionality, and use cases. In this essay, we will explore the differences between LangChain and LangSmith, highlighting their strengths, weaknesses, and applications.

**Overview of LangChain**

LangChain is an open-source framework that enables users to build custom AI applications using a modular architecture. This framework allows users to create custom AI models and integrate them with various data sources, making it a highly flexible and scalable tool. LangChain's modular architecture enables users to build